In [1]:
import os
import sys
import torch
import radiomics
import torchio as tio
import SimpleITK as sitk
import numpy as np

from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from radiomics.featureextractor import RadiomicsFeatureExtractor
from radiomics.imageoperations import checkMask

import logging
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)

In [2]:
# imgs_interobserver = sorted([file for file in glob("../../data/NSCLC-Interobserver/volume+seg/*image*")])
# masks_interobserver = sorted([file for file in glob("../../data/NSCLC-Interobserver/volume+seg/*segmentation*")])

# imgs_radiomics = sorted([file for file in glob("../../data/NSCLC-Radiomics/volume+seg/*image*")])
# masks_radiomics = sorted([file for file in glob("../../data/NSCLC-Radiomics/volume+seg/*segmentation*")])

# imgs_radiogenomics = sorted([file for file in glob("../../data/NSCLC-Radiogenomics/volume+seg/*image*")])
# masks_radiogenomics = sorted([file for file in glob("../../data/NSCLC-Radiogenomics/volume+seg/*segmentation*")])

# images = imgs_interobserver+imgs_radiomics+imgs_radiogenomics
# masks = masks_interobserver+masks_radiomics+masks_radiogenomics

# print(len(images), len(masks))

# assert len(images) == len(masks)

# subjects = [filename.split("/")[-1].split("_")[0] for filename in images]

# all_masks_exist = True
# for file in masks:
#     if not os.path.exists(file):
#         print(f"Mask does not exsist: {file}")
#         all_masks_exist = False

# if all_masks_exist:
#     print("All masks exist")

# all_imgs_exist = True
# for file in images:
#     if not os.path.exists(file):
#         print(f"Image does not exsist: {file}")
#         all_imgs_exist = False

# if all_imgs_exist:
#     print("All imgs exist")

In [3]:
all_filepaths = glob("/media/johannes/WD Elements/NSCLC_Stefan/median_resampled+max_cropped/*image.nii.gz")
all_labels = glob("/media/johannes/WD Elements/NSCLC_Stefan/labels/*")
all_labels_ids = [label.split("/")[-1].split("_")[0] for label in all_labels]

images = sorted([file for file in all_filepaths if file.split("/")[-1].split("_")[0] in all_labels_ids])
masks = [img.replace("image", "label") for img in images]
subjects = [filename.split("/")[-1].split("_")[0] for filename in images]

files_ids = [file.split("/")[-1].split("_")[0] for file in images]

labels = sorted([label for label in all_labels if label.split("/")[-1].split("_")[0] in files_ids])
labels_binary = [0 if "adeno" in label else 1 if "squamous" in label else np.nan for label in labels]

assert len(images) == len(masks) == len(subjects) == len(labels_binary)!= 0

In [4]:
feature_list = []
subjects_list = []
label_list = []

paramFile = os.path.abspath("./Params-NSCLC.yaml")

feature_extractor = RadiomicsFeatureExtractor(paramFile)
# resample = tio.transforms.Resample(target=1.0)

for idx in tqdm(range(len(images))):       
    
    # try:
    img = tio.ScalarImage(images[idx])
    seg = tio.transforms.Resample(target=img)(tio.LabelMap(masks[idx]))
    
    subject = tio.Subject(volume=img, segmentation=seg)

    # subject = resample(subject)
    features = feature_extractor.execute(subject.volume.as_sitk(), subject.segmentation.as_sitk(), label=1)
    feature_list.append(features)
    subjects_list.append(subjects[idx])
    label_list.append(labels_binary[idx])

    # except Exception as e:
    #     print(e)        
    #     print(images[idx])
    #     continue

torch.save(feature_list, f"./NSCLC_features.pt")
torch.save(subjects_list, f"./NSCLC_subjects.pt")
torch.save(label_list,  f"./NSCLC_labels.pt")

  0%|          | 0/359 [00:00<?, ?it/s]

100%|██████████| 359/359 [08:57<00:00,  1.50s/it]


In [4]:
len(features.keys())

843

In [5]:
original_features = []
wavelet_features = []
others = []

for feature in features.keys():
    if feature.startswith("original"):
        original_features.append(feature)

    elif feature.startswith("wavelet"):
        wavelet_features.append(feature)

    else:
        others.append(feature)

In [6]:
len(original_features)

102

In [7]:
len(wavelet_features)

704

In [8]:
len(original_features) + len(wavelet_features)

806